In [1]:
import boto3
import pandas as pd
import numpy as np

In [2]:
def download_file_from_s3(bucket, object_name, local_file_name):
    """
    Download a file from S3 to the local file system.

    :param bucket: Name of the S3 bucket
    :param object_name: S3 object name
    :param local_file_name: Local file name to save the downloaded file
    """
    # # When using IAM roles, boto3 retrieves credentials from the instance metadata
    # s3_client = boto3.client('s3')

    #When setting up credentials locally, use the following code
    session = boto3.Session()
    s3_client = session.client('s3')

    s3_client.download_file(bucket, object_name, local_file_name)

bucket_name = 'myukdata'
s3_file_name = 'Original/Member/Member_ElectionsContested.csv'
local_file = 'Member_ElectionsContested.csv'

download_file_from_s3(bucket_name, s3_file_name, local_file)

In [3]:
df = pd.read_csv('Member_ElectionsContested.csv')
df.info()
pd.set_option('display.max_rows', None)
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2506 entries, 0 to 2505
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   memberID            2506 non-null   int64  
 1   house               2506 non-null   int64  
 2   name                2506 non-null   object 
 3   id                  2506 non-null   int64  
 4   startDate           2506 non-null   object 
 5   endDate             0 non-null      float64
 6   additionalInfo      0 non-null      float64
 7   additionalInfoLink  0 non-null      float64
dtypes: float64(3), int64(3), object(2)
memory usage: 156.8+ KB


memberID  house                       name    id            startDate  \
0         1      1         North Bedfordshire  2105  1992-04-09T00:00:00   
1         2      1                Luton North  2146  1983-06-09T00:00:00   
2         3      1  Carshalton and Wallington   608  1992-04-09T00:00:00   
3         4      1               Bristol East   463  1992-04-09T00:00:00   
4         5      1            Lambeth Central  1673  1974-10-10T00:00:00   

   endDate  additionalInfo  additionalInfoLink  
0      NaN             NaN                 NaN  
1      NaN             NaN                 NaN  
2      NaN             NaN                 NaN  
3      NaN             NaN                 NaN  
4      NaN             NaN                 NaN

In [4]:
df.drop(columns=['endDate',
                 'additionalInfo',
                 'additionalInfoLink',
                 'house'], inplace=True)

df.rename(columns={'memberID': 'memberId',
                     'name': 'constituencyName',
                     'id':'constituencyId'}, inplace=True)
df['startDate'] = pd.to_datetime(df['startDate'])


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2506 entries, 0 to 2505
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   memberId          2506 non-null   int64         
 1   constituencyName  2506 non-null   object        
 2   constituencyId    2506 non-null   int64         
 3   startDate         2506 non-null   datetime64[ns]
dtypes: datetime64[ns](1), int64(2), object(1)
memory usage: 78.4+ KB


In [6]:
#Save the file to S3
import boto3
import io

def upload_df_to_s3(df, bucket, object_name):
    """
    Upload a DataFrame to an S3 bucket as CSV.

    :param df: DataFrame to upload
    :param bucket: Bucket to upload to
    :param object_name: S3 object name
    :return: True if the DataFrame was uploaded, else False
    """
    # Create a buffer
    csv_buffer = io.StringIO()
    df.to_csv(csv_buffer, index=False)

    # Move to the start of the buffer
    csv_buffer.seek(0)

    # Upload the buffer content to S3
    s3_client = boto3.client('s3')
    try:
        s3_client.put_object(Bucket=bucket, Key=object_name, Body=csv_buffer.getvalue())
    except ClientError as e:
        logging.error(e)
        return False
    return True

bucket_name = 'myukdata'
object_name = 'Preprocessing/Member/MemberElectionContested.csv'
upload_df_to_s3(df, bucket_name, object_name)


True